TP 4 : Exercice 2



In [ ]:
! pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("/content/drive/MyDrive/TP04_data").getOrCreate()

1)	Stocker le contenu de tous les fichiers du dossier exo2 dans une variable.

In [ ]:
staticDataFrame = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/content/drive/MyDrive/TP04_data/*.csv")

2)	Afficher le schéma du Dataframe obtenu.

In [ ]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
staticDataFrame.collect()

[Row(InvoiceNo='537226', StockCode='22811', Description='SET OF 6 T-LIGHTS CACTI ', Quantity=6, InvoiceDate=datetime.datetime(2010, 12, 6, 8, 34), UnitPrice=2.95, CustomerID=15987.0, Country='United Kingdom'),
 Row(InvoiceNo='537226', StockCode='21713', Description='CITRONELLA CANDLE FLOWERPOT', Quantity=8, InvoiceDate=datetime.datetime(2010, 12, 6, 8, 34), UnitPrice=2.1, CustomerID=15987.0, Country='United Kingdom'),
 Row(InvoiceNo='537226', StockCode='22927', Description='GREEN GIANT GARDEN THERMOMETER', Quantity=2, InvoiceDate=datetime.datetime(2010, 12, 6, 8, 34), UnitPrice=5.95, CustomerID=15987.0, Country='United Kingdom'),
 Row(InvoiceNo='537226', StockCode='20802', Description='SMALL GLASS SUNDAE DISH CLEAR', Quantity=6, InvoiceDate=datetime.datetime(2010, 12, 6, 8, 34), UnitPrice=1.65, CustomerID=15987.0, Country='United Kingdom'),
 Row(InvoiceNo='537226', StockCode='22052', Description='VINTAGE CARAVAN GIFT WRAP', Quantity=25, InvoiceDate=datetime.datetime(2010, 12, 6, 8, 34)

3) Remplir les valeurs manquantes (NaN) avec la valeur 0.

4) Ajouter une nouvelle colonne nommée "day_of_week". La valeur de la colonne est le jour de la semaine correspondant à la date de chaque ligne dans la colonne
"InvoiceDate".

In [ ]:
from pyspark.sql.functions import date_format, col
preppedDataFrame = staticDataFrame\
.na.fill(0)\
.withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
#.coalesce(5)


5)	Diviser les données en un ensemble d'apprentissage et un ensemble de test. Effectuer la division en se basant sur l'attribut InvoiceDate : l'ensemble d'apprentissage contient les achats effectués avant 2012-12-13 et l'ensemble de test contient les achats effectués durant ou après 2012-12-13.

In [ ]:
trainDataFrame = preppedDataFrame\
.where("InvoiceDate < '2010-12-13'")
testDataFrame = preppedDataFrame\
.where("InvoiceDate >= '2010-12-13'")

In [ ]:
print(trainDataFrame.count())
print(testDataFrame.count())

26732
18676


6)	Créer un StringIndexer qui permet de transformer les jours de semaine présents dans la colonne day_of_week en valeurs numériques correspondantes.

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
.setInputCol("day_of_week")\
.setOutputCol("day_of_week_index")

7)	En utilisant le StringIndexer, Spark peut par exemple représenter samedi par 6 et lundi par 1. Cependant, avec ce schéma de numérotation, nous indiquons implicitement que samedi est supérieur à lundi (par des valeurs numériques pures). Ceci est évidemment incorrect. Comment résoudre ce problème ?

In [ ]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
.setInputCol("day_of_week_index")\
.setOutputCol("day_of_week_encoded")

8)	Créer un VectorAssembler contenant trois attributs : UnitPrice, Quantity, et day_of_week_encoded.

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler()\
.setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
.setOutputCol("features")

9)	Créer un pipeline configuré avec les résultats des étapes 6, 7 et 8.

In [ ]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline()\
.setStages([indexer, encoder, vectorAssembler])

10)	Notre StringIndexer doit savoir combien de valeurs uniques il y a à indexer, comment résoudre ce problème ?

In [ ]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

11)	Transformer les données de l’ensemble d’apprentissage en se basant sur les étapes (stages) du pipeline.

In [ ]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

12)	Créer une instance de KMeans, on supposse que le nombre de clusters est 20.

In [ ]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
.setK(20)

13)	Lancer l’apprentissage de KMeans en se basant sur l’ensemble obtenu dans l’étape 11.

In [ ]:
kmModel = kmeans.fit(transformedTraining)

14)	Effectuer des prédictions sur l’ensemble de test.

In [ ]:
transformedTest = fittedPipeline.transform(testDataFrame)
predictions = kmModel.transform(transformedTest)

15)	Calculer le coefficient de silhouette.

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)